<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/ResNet152%20with%20hidden%20dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 31.8 MB/s 
     |████████████████████████████████| 182 kB 67.5 MB/s 
     |████████████████████████████████| 166 kB 71.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 59.4 MB/s 
     |████████████████████████████████| 162 kB 78.1 MB/s 
     |████████████████████████████████| 162 kB 79.6 MB/s 
     |████████████████████████████████| 158 kB 75.8 MB/s 
     |████████████████████████████████| 157 kB 78.8 MB/s 
     |████████████████████████████████| 157 kB 79.9 MB/s 
     |████████████████████████████████| 157 kB 70.1 MB/s 
     |████████████████████████████████| 157 kB 77.2 MB/s 
     |████████████████████████████████| 157 kB 77.5 MB/s 
     |████████████████████████████████| 157 kB 71.4 MB/s 
     |████████████████████████████████| 157 kB 72.4 MB/s 
     |████████████████████████████████| 156 kB 78.2 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [12]:
resnet152 = tf.keras.applications.ResNet152(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg')
model_resnet152=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet152,
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
resnet152.trainable=False
model_resnet152.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 resnet152 (Functional)      (None, 2048)              58370944  
                                                                 
 dense_2 (Dense)             (None, 32)                65568     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 5)                 165       
                                                                 
Total params: 58,436,677
Trainable params: 65,733
Non-trainable params: 58,370,944
_________________________________________________________________


In [13]:
model_resnet152.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [14]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "resnet152 with hidden dense",
)

In [15]:
%%time
epochs=40
history = model_resnet152.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.4025 - accuracy: 0.4048

wandb: Adding directory to artifact (/content/wandb/run-20221105_071626-1o1hc94b/files/model-best)... Done. 1.6s


22/22 [==============================] - 202s 9s/step - loss: 1.4025 - accuracy: 0.4048 - val_loss: 1.3419 - val_accuracy: 0.5371
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.3014 - accuracy: 0.5298

wandb: Adding directory to artifact (/content/wandb/run-20221105_071626-1o1hc94b/files/model-best)... Done. 1.5s


22/22 [==============================] - 70s 3s/step - loss: 1.3014 - accuracy: 0.5298 - val_loss: 1.2822 - val_accuracy: 0.5086
Epoch 3/40
22/22 [==============================] - ETA: 0s - loss: 1.2648 - accuracy: 0.5256

...

22/22 [==============================] - 70s 3s/step - loss: 0.9938 - accuracy: 0.6065 - val_loss: 1.0288 - val_accuracy: 0.6057
Epoch 36/40
22/22 [==============================] - ETA: 0s - loss: 0.9959 - accuracy: 0.5980

wandb: Adding directory to artifact (/content/wandb/run-20221105_071626-1o1hc94b/files/model-best)... Done. 1.5s


22/22 [==============================] - 75s 4s/step - loss: 0.9959 - accuracy: 0.5980 - val_loss: 1.0263 - val_accuracy: 0.5429
Epoch 37/40
22/22 [==============================] - ETA: 0s - loss: 0.9812 - accuracy: 0.5966

wandb: Adding directory to artifact (/content/wandb/run-20221105_071626-1o1hc94b/files/model-best)... Done. 1.5s


22/22 [==============================] - 71s 3s/step - loss: 0.9812 - accuracy: 0.5966 - val_loss: 1.0062 - val_accuracy: 0.5429
Epoch 38/40
22/22 [==============================] - 8s 348ms/step - loss: 0.9614 - accuracy: 0.6165 - val_loss: 1.0183 - val_accuracy: 0.5714
Epoch 39/40
22/22 [==============================] - 8s 350ms/step - loss: 0.9685 - accuracy: 0.6236 - val_loss: 1.0234 - val_accuracy: 0.5429
Epoch 40/40
22/22 [==============================] - 8s 354ms/step - loss: 0.9832 - accuracy: 0.6136 - val_loss: 1.0213 - val_accuracy: 0.6171
CPU times: user 23min 4s, sys: 1min 25s, total: 24min 30s
Wall time: 32min 33s


In [16]:
wandb.finish()

accuracy,▁▅▅▄▆▆▆▆▆▆▇▆▆▇▇▆▆▇▇▇▇▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇███
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▆▆▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁
val_accuracy,▃▁▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▆▅▄▃▃▄▃▂▃▂▆█▇▃▃▅▃█
val_loss,█▇▆▆▅▅▄▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
accuracy,0.61364
best_epoch,36
best_val_loss,1.00617
epoch,39
loss,0.98321
val_accuracy,0.61714


In [17]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
